Tutorial walkthrough (https://medium.freecodecamp.org/sqlalchemy-makes-etl-magically-easy-ab2bd0df928)

#### Install SQLalchemy

`$ pip install sqlalchemy`

# WHAT IS ETL process?

### Extract - from our datasource
### Transform - into appropriate format in order to store in our DB
### Load - the data into our DB

* * * *

### Defininig Schema

database schema defines the structure of a dabase system
- tables
- columns
- fields
- the relationships between them

Schemas can be defined in raw SQL, or through the use of SQLAlchemy's ORM feature

* * * *

#### (1) Import SQLAlchemy

In [1]:
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import *


import pickle

#### (2) Connect to database

For engine configuration for different Database, check the link below  
<http://docs.sqlalchemy.org/en/latest/core/engines.html>

In [2]:
pw = pickle.load(open("sqlalchemy_mysql_pw.pickle", "rb"))

In [3]:
engine = create_engine("mysql+mysqldb://root:" + pw + "@52.78.44.120/test_alchemy")

#### (3) Define table classes

- format --> `<ColumnName> = Column(type)`

`type` is a data type *such as Interger, String, Datetime and so on.  
Use `primary_key=True` to denote columns which will be used as primary keys

In [4]:
Base = declarative_base()

In [5]:
class Users(Base):
    __tablename__ = "users"
    UserId = Column(Integer, primary_key=True)
    Title = Column(VARCHAR(255))
    FirstName = Column(VARCHAR(255))
    LastName = Column(VARCHAR(255))
    Email = Column(VARCHAR(255))
    UserName = Column(VARCHAR(255))
    DOB = Column(DateTime)

In [6]:
class Uploads(Base):
    __tablename__ = "uploads"
    UploadId = Column(Integer, primary_key=True)
    UserId = Column(Integer)
    Title = Column(VARCHAR(255))
    Body = Column(VARCHAR(255))
    Timestamp = Column(DateTime)

#### (4) Create tables.

`checkfirst=True` parameter ensures that new tables are only created if they do not currently exist in the database

In [7]:
Users.__table__.create(bind=engine, checkfirst=True)
Uploads.__table__.create(bind=engine, checkfirst=True)

* * * *

### Extract

Once the schema has been defined, the next task is Extraction of raw data from its source

In [8]:
import requests

The example data is held in two objects in JSON format

In [9]:
url = 'https://randomuser.me/api/?results=10'
users_json = requests.get(url).json()

In [10]:
url2 = 'https://jsonplaceholder.typicode.com/posts/'
uploads_json = requests.get(url2).json()

In [11]:
users_json

{'info': {'page': 1,
  'results': 10,
  'seed': '9eee90dcd7c3ba33',
  'version': '1.1'},
 'results': [{'cell': '(267)-965-7640',
   'dob': '1989-11-30 10:01:50',
   'email': 'dwight.parker@example.com',
   'gender': 'male',
   'id': {'name': 'SSN', 'value': '194-66-8550'},
   'location': {'city': 'buffalo',
    'postcode': 50614,
    'state': 'nevada',
    'street': '5579 bruce st'},
   'login': {'md5': '40cf5ed27da942bda80f6b467c89c5aa',
    'password': 'rugby1',
    'salt': 'Qo0IRzUs',
    'sha1': 'a2badb4e585c430f249609a8f9b48a65fbfc498a',
    'sha256': 'e42afa5b4ccc3d37f0d4145ce731784309dd9746a16fe45a4ee672165672d7e4',
    'username': 'greenpanda303'},
   'name': {'first': 'dwight', 'last': 'parker', 'title': 'mr'},
   'nat': 'US',
   'phone': '(924)-134-8657',
   'picture': {'large': 'https://randomuser.me/api/portraits/men/15.jpg',
    'medium': 'https://randomuser.me/api/portraits/med/men/15.jpg',
    'thumbnail': 'https://randomuser.me/api/portraits/thumb/men/15.jpg'},
   'regi

In [12]:
uploads_json

[{'body': 'quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto',
  'id': 1,
  'title': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit',
  'userId': 1},
 {'body': 'est rerum tempore vitae\nsequi sint nihil reprehenderit dolor beatae ea dolores neque\nfugiat blanditiis voluptate porro vel nihil molestiae ut reiciendis\nqui aperiam non debitis possimus qui neque nisi nulla',
  'id': 2,
  'title': 'qui est esse',
  'userId': 1},
 {'body': 'et iusto sed quo iure\nvoluptatem occaecati omnis eligendi aut ad\nvoluptatem doloribus vel accusantium quis pariatur\nmolestiae porro eius odio et labore et velit aut',
  'id': 3,
  'title': 'ea molestias quasi exercitationem repellat qui ipsa sit aut',
  'userId': 1},
 {'body': 'ullam et saepe reiciendis voluptatem adipisci\nsit amet autem assumenda provident rerum culpa\nquis hic commodi nesciunt rem tenetur dolore

* * * *

### Transform

It is important to ensure that it is in the correct format. The JSON objects created in the code above are nested, and contain more data than is requred fro the tables defined

Intermediary step **Data Tranformation** is important for the current nested JSONN format, so that the flat format can be safely written to the database without error

The code below creates two lists, `users` and `uploads`

In [13]:
from datetime import datetime, timedelta
from random import randint

In [14]:
users = []
uploads = []

In [15]:
for i, result in enumerate(users_json['results']):
    row = {}
    row['UserId'] = i
    row['Title'] = result['name']['title']
    row['FirstName'] = result['name']['first']
    row['LastName'] = result['name']['last']
    row['Email'] = result['email']
    row['UserName'] = result['login']['username']
    dob = datetime.strptime(result['dob'], '%Y-%m-%d %H:%M:%S')
    row['DOB'] = dob.date()
    users.append(row)

In [16]:
for result in uploads_json:
    row = {}
    row['UploadId'] = result['id']
    row['UserId'] = result['userId']
    row['Title'] = result['title']
    row['Body'] = result['body']
    delta = timedelta(seconds=randint(1, 86400))
    row['Timestamp'] = datetime.now() - delta
    uploads.append(row)

In [21]:
users[0]['UserId'] = 10

The main goal for the for loops is to iterate through the JSON objects. For each result, create a new Python dictionary object with keys corresponding to each column defined for the relevant table in the schema. This ensures that the data is no longer nested but keeps only the data we need for the database tables

The other step is to use Python's `datetime` module to parse dates and transform them into `DateTime` type objects that can be written to the databse. For the sake of this example, random `DataTime` objects are generated using the timedelta() method.

Each creaed dictionary is appended to a list

* * * *

### Load

Finally, the data is in a form that can be loaded into the database. SQLAlchemy makes this step straightforward through its Session API

The Session API acts a bit like a middleman, or "holding zone," for Python objects you have either loaded from or associated with the database. These objects can be manipulated within the session before being committed to the database:

`sessionmaker` is used to generate newly-configured `Session` classes

In [22]:
Session = sessionmaker(bind=engine)
session = Session()

Next up are two loops which iterate through the `users` and `uploads` lists. The dictionary objects whose keys correspond to the columns given in the `Users` and `Uploads` classes defined previously. 

Each object is used to instantiate a new instance of the relevant class (using Python's `some_function(**some_dict**)` trick). The object is added to the current session with `session.add()`

In [23]:
for user in users:
    row = Users(**user)
    session.add(row)

In [24]:
for upload in uploads:
    row = Uploads(**upload)
    session.add(row)

In [25]:
session.commit()

Finally, when the session contains the rwos to be added, `session.commit()` is used to commit the transaction to the database.

* * * *

Different versions of SQL have somewhat incompatible syntaxes, but AQLAlchemy's Expression Language acts as a lingua franca between them.

Also, being able to query and interact with your database in a Python is a real advantage to developers whod'prefer working with Python. However, SQLAlchemy also lets you work in plain SQL, for cases when it is simpler to use a pre-written query.

### Aggregating

Here, we'll create an aggregarted table, showing how many articles each user has posted, and the time they were last active.

In [26]:
class UploadCounts(Base):
    __tablename__ = "upload_counts"
    UserId = Column(Integer, primary_key=True)
    LastActive = Column(DateTime)
    PostCount = Column(Integer)

In [27]:
UploadCounts.__table__.create(bind=engine, checkfirst=True)

In plain SQL, this table would be poplulated using a query along th elines of :

```
INSERT INTO upload_counts
SELECT
    UserId,
    MAX(Timestamp) AS LastActive,
    COUNT(UploadId) AS PostCount
FROM
    uploads
GROUP BY 1;
```


In [28]:
connection = engine.connect()

The query is defined using the select() function

This query is the same as the plain SQL version given above. 
- Selects the `UserId` column from the uploads table. 
- Applies `func.max()` to the Timestamp column, which identifies the most recent timestamp. 
    - Labelled `LastActive` using the label() method.
- Applies `func.count()` to count the number of records that appear in the Title column. 
    - Labelled `PostCount` using the label() method.
- Uses `group_by()` to group results by `UserID`

In [38]:
query = select([Uploads.UserId,
               func.max(Uploads.Timestamp).label('LastActive'),
               func.count(Uploads.UploadId).label('PostCount')]).\
               group_by('UserId')

To use the results of the query, a for loop iterates over the row objects returned by `connection.execute(query)`. 

In [39]:
results = connection.execute(query)

In [40]:
for result in results:
    row = UploadCounts(**result)
    session.add(row)

Each row is added to the `session` object, and finally the session is committed to the databse

In [41]:
session.commit()

In [42]:
session.close()

* * * *